In [1]:
import json
import numpy as np
import pandas as pd
import plotly.express as px

In [2]:
with open('../../Application/bin/data/Patch.json') as file:
	patch = json.load(file)
	

In [3]:
def find_node_by_name(node_name):
	for node_id in patch['Nodes']:
		if patch['Nodes'][node_id]['Name'] == node_name:
			return patch['Nodes'][node_id]
	raise(Exception("Node not found"))

In [4]:
calibrate_node = find_node_by_name('Calibrate')

In [5]:
first_camera = calibrate_node['Content']['cameraCaptures']['captures'][0]

In [6]:
#gather laser image points function
def gather_laser_image_points(camera_capture):
	laser_image_points = []
	laser_names = []
	for laser_capture in first_camera['laserCaptures']['captures']:
		if laser_capture['Selected']:
			laser_image_points.append(laser_capture['imagePointInCamera'])
			laser_names.append(laser_capture['laserAddress'])
	return laser_image_points, laser_names

In [7]:
laser_image_points, laser_names = gather_laser_image_points(first_camera)
data = np.array(laser_image_points)

In [8]:
fig = px.scatter(x=data[:,0], y=data[:,1],hover_name=laser_names)
fig.show()

In [9]:
def get_laser_capture(camera_capture, laserAddress):
	for laser_capture in camera_capture['laserCaptures']['captures']:
		if int(laser_capture['laserAddress']) == laserAddress:
			return laser_capture

get_laser_capture(first_camera, 22).keys()

dict_keys(['Color', 'Selected', 'beamCaptures', 'date', 'directory', 'imagePointInCamera', 'laserAddress', 'time', 'timestamp'])

In [11]:
# gather data
data_points = []
for camera_capture in calibrate_node['Content']['cameraCaptures']['captures']:
	for laser_capture in camera_capture['laserCaptures']['captures']:
		if laser_capture['Selected']:
			data_point = {}
			data_point['Camera'] = "{0} {1}".format(camera_capture['date'], camera_capture['time'])
			data_point['x'] = laser_capture['imagePointInCamera'][0]
			data_point['y'] = laser_capture['imagePointInCamera'][1]
			data_point['Laser'] = "Laser {0}".format(laser_capture['laserAddress'])
			
			if 'residual' in data_point:
				data_point['residual'] = laser_capture['laserAddress']
			
			#count active beam captures
			active_beam_capture_count = 0
			for beam_capture in laser_capture['beamCaptures']['captures']:
				if beam_capture['Selected']:
					active_beam_capture_count = active_beam_capture_count + 1
			data_point['Active beam captures'] = active_beam_capture_count
			
			data_points.append(data_point)



In [16]:
fig = px.scatter(data_points, x='x', y='y', hover_name='Laser', hover_data=['Active beam captures'])
fig.show()

In [15]:
data_points

[{'Camera x': 4132.10107421875,
  'Camera y': 7955.50927734375,
  'Laser': 'Laser 1',
  'Active beam captures': 9},
 {'Camera x': 5506.77587890625,
  'Camera y': 7905.3994140625,
  'Laser': 'Laser 3',
  'Active beam captures': 9},
 {'Camera x': 6176.28857421875,
  'Camera y': 7906.13037109375,
  'Laser': 'Laser 4',
  'Active beam captures': 9},
 {'Camera x': 6745.06689453125,
  'Camera y': 7894.9892578125,
  'Laser': 'Laser 5',
  'Active beam captures': 9},
 {'Camera x': 7284.85302734375,
  'Camera y': 7880.24560546875,
  'Laser': 'Laser 6',
  'Active beam captures': 9},
 {'Camera x': 8019.029296875,
  'Camera y': 7848.1748046875,
  'Laser': 'Laser 7',
  'Active beam captures': 9},
 {'Camera x': 8601.16796875,
  'Camera y': 7814.1611328125,
  'Laser': 'Laser 8',
  'Active beam captures': 9},
 {'Camera x': 10029.76171875,
  'Camera y': 7687.11669921875,
  'Laser': 'Laser 10',
  'Active beam captures': 9},
 {'Camera x': 10799.8349609375,
  'Camera y': 7591.68603515625,
  'Laser': 'Laser 

In [21]:
calibrate_node['Content']['cameraCaptures']['captures'][0]['date']

'Fri 2021.10.01'